시작하자

In [41]:

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 
import seaborn as sns #시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime

import os
print(os.listdir("./input"))

['sampleSubmission.csv', 'test.csv', 'train.csv']


In [42]:
#훈련데이터와 테스트 데이터 세트를 불러온다
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [44]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [45]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [46]:

train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [47]:
# datetime을 년,월,일,시 값으로 나누어서 확인
train['year'] = train.datetime.dt.year
train['month'] = train.datetime.dt.month
train['day'] = train.datetime.dt.day
train['hour'] = train.datetime.dt.hour
train['dayofweek'] = train.datetime.dt.dayofweek

test['year'] = test.datetime.dt.year
test['month'] =test.datetime.dt.month
test['day'] = test.datetime.dt.day
test['hour'] = test.datetime.dt.hour
test['dayofweek'] = test.datetime.dt.dayofweek

In [48]:
# datetime 값에서 얻을수 있는거 다 얻었으니 지운다
train = train.drop('datetime',axis=1)
test = test.drop('datetime',axis=1)

In [49]:
# windspeed에 0 값이 많다 삭제하자
train = train.drop('day',axis=1)
test = test.drop('day',axis=1)
train = train.drop('month',axis=1)
test = test.drop('month',axis=1)

In [50]:
# casual, registered 날리자
train = train.drop('casual',axis=1)
train = train.drop('registered',axis=1)


In [51]:
train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,hour,dayofweek
0,1,0,0,1,9.84,14.395,81,0.0,16,2011,0,5
1,1,0,0,1,9.02,13.635,80,0.0,40,2011,1,5
2,1,0,0,1,9.02,13.635,80,0.0,32,2011,2,5
3,1,0,0,1,9.84,14.395,75,0.0,13,2011,3,5
4,1,0,0,1,9.84,14.395,75,0.0,1,2011,4,5


In [52]:
test.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,hour,dayofweek
0,1,0,1,1,10.66,11.365,56,26.0027,2011,0,3
1,1,0,1,1,10.66,13.635,56,0.0000,2011,1,3
2,1,0,1,1,10.66,13.635,56,0.0000,2011,2,3
3,1,0,1,1,10.66,12.880,56,11.0014,2011,3,3
4,1,0,1,1,10.66,12.880,56,11.0014,2011,4,3


모델 평가

In [53]:
# y is predict value y_ is actual value
def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y), 
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

In [15]:
from sklearn.model_selection import train_test_split
target = train['count']
features = train.drop(columns=['count'])
train_input, test_input, train_target, test_target = train_test_split(
    features,target, random_state=42
)

train_target_log=np.log1p(train_target)
test_target_log = np.log1p(test_target)

선형회귀

In [16]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
lr = LinearRegression()

lr.fit(train_input,train_target_log)
print(lr.score(train_input,train_target_log))
print(lr.score(test_input,test_target_log))
# 성능평가
pred = lr.predict(test_input)
print(rmsle(np.exp(test_target_log),np.exp(pred),False))

0.4897471682664274
0.4790748778123245
0.983621968597669


In [18]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

rf.fit(train_input,train_target_log)
print(rf.score(train_input,train_target_log))
print(rf.score(test_input,test_target_log))
preds = rf.predict(test_input)
print('RMSLE Value for RandomForest {}'.format(rmsle(np.exp(test_target_log),np.exp(preds),False)))

0.9936556719087887
0.9574408994342695
RMSLE Value for RandomForest 0.27444832150660214


히스토그램 기반 그래디언트

In [55]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
target = train['count']
features = train.drop(columns=['count'])
target_log = np.log1p(target)
train_input,test_input,train_target,test_target = train_test_split(features,target_log,random_state=42)

hgb = HistGradientBoostingRegressor()
hgb.fit(train_input,train_target)
print(hgb.score(train_input,train_target))
print(hgb.score(test_input,test_target))
preds = hgb.predict(train_input)
print('RMSLE Value {}'.format(rmsle(np.exp(train_target),np.exp(preds),False)))

0.9692585338035657
0.957228004492078
RMSLE Value 0.2339629206812164


0.38269 이것도 많이 떨어졌네

In [54]:
from sklearn.ensemble import GradientBoostingRegressor
target = train['count']
features = train.drop(columns=['count'])
target_log = np.log1p(target)
train_input,test_input,train_target,test_target = train_test_split(features,target_log,random_state=42)

gb1 = GradientBoostingRegressor(n_estimators=1500,alpha=0.02)

gb1.fit(train_input,train_target)
print(gb1.score(train_input,train_target))
print(gb1.score(test_input,test_target))
preds = gb1.predict(train_input)
print('RMSLE Value {}'.format(rmsle(np.exp(train_target),np.exp(preds),False)))

0.9722502924156528
0.957025640112616
RMSLE Value 0.22154481912201002


0.37737 많이 떨어졌네

In [58]:
# 성능평가
pred = hgb.predict(test)

print(pred.shape)

# 다시 지수로 바꾸기
pred_exp = np.exp(pred)
# evaluate_regr(test_target_exp,pred_exp)

pred_exp

(6493,)


array([ 11.09630604,   6.02549158,   3.98924714, ..., 105.14303781,
        80.5975563 ,  48.96725479])

In [59]:
submit = pd.read_csv("./input/sampleSubmission.csv")
submit['count'] = pred_exp
submit.to_csv("./results_hgb222.csv",index=False)

0.44136점 757등 23.35%

1.0v 에서 모델을 추가해봤다.

히스토그램 기반 그레디언트 모델로 바꾼후 0.41009 로 좋아졌다. 337등